In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import (
    ResNet50, ResNet101, ResNet152,
    EfficientNetB0, EfficientNetB3, EfficientNetB7,
    DenseNet121, DenseNet169, DenseNet201,


In [ ]:
    InceptionV3, InceptionResNetV2,
    MobileNetV2, MobileNetV3Large,
    VGG16, VGG19,
    Xception, NASNetLarge
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,
    TensorBoard, LearningRateScheduler, CSVLogger
)

# Sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    accuracy_score, precision_recall_fscore_support,
    roc_auc_score, cohen_kappa_score, matthews_corrcoef
)
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

# Others
from collections import Counter
import json
from datetime import datetime
import pickle

In [2]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# GPU Configuration
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print(f"✓ GPU Available: {physical_devices}")
    except:
        print("✓ GPU Available but memory growth setting failed")
else:
    print("⚠ Running on CPU")

✓ GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# ============================================================================
# 1. DATA LOADING AND EXPLORATION
# ============================================================================

class SoilDataLoader:
    """Advanced data loader with multiple format support"""
    
    def __init__(self, data_path):
        self.data_path = Path(data_path)
        self.data_type = None
        self.images = []
        self.labels = []
        self.class_names = []
        
    def load_image_directory(self, img_size=(224, 224)):
        """Load images directly from class folders"""
        self.class_names = sorted([d.name for d in self.data_path.iterdir() if d.is_dir()])
        print(f"✓ Found {len(self.class_names)} classes: {self.class_names}")
        
        for class_idx, class_name in enumerate(self.class_names):
            class_path = self.data_path / class_name
            image_files = list(class_path.glob('*.jpg')) + \
                         list(class_path.glob('*.png')) + \
                         list(class_path.glob('*.jpeg')) + \
                         list(class_path.glob('*.JPG'))
            
            print(f"   Loading {class_name}: {len(image_files)} images...")
            
            for img_path in image_files:
                try:
                    img = cv2.imread(str(img_path))
                    if img is None:
                        continue
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, img_size)
                    self.images.append(img)
                    self.labels.append(class_idx)
                except Exception as e:
                    continue
        
        self.images = np.array(self.images)
        self.labels = np.array(self.labels)
        print(f"✓ Successfully loaded {len(self.images)} images")
        return self.images, self.labels, self.class_names


class SoilEDA:
    """Comprehensive Exploratory Data Analysis"""
    
    def __init__(self, images, labels, class_names):
        self.images = images
        self.labels = labels
        self.class_names = class_names
        self.results = {}
        
    def analyze_all(self):
        """Run all EDA analyses"""
        print("\n" + "="*70)
        print("EXPLORATORY DATA ANALYSIS")
        print("="*70)
        
        self.basic_statistics()
        self.class_distribution()
        self.image_properties()
        self.color_analysis()
        self.texture_analysis()
        self.generate_report()
        
    def basic_statistics(self):
        """Basic dataset statistics"""
        print("\n📊 BASIC STATISTICS")
        print("-" * 50)
        print(f"Total samples: {len(self.images)}")
        print(f"Number of classes: {len(self.class_names)}")
        print(f"Image shape: {self.images[0].shape}")
        print(f"Data type: {self.images.dtype}")
        print(f"Value range: [{self.images.min()}, {self.images.max()}]")
        
        self.results['total_samples'] = len(self.images)
        self.results['num_classes'] = len(self.class_names)
        
    def class_distribution(self):
        """Analyze class distribution and imbalance"""
        print("\n📈 CLASS DISTRIBUTION")
        print("-" * 50)
        
        class_counts = Counter(self.labels)
        for idx, count in sorted(class_counts.items()):
            print(f"{self.class_names[idx]}: {count} ({count/len(self.labels)*100:.2f}%)")
        
        max_count = max(class_counts.values())
        min_count = min(class_counts.values())
        imbalance_ratio = max_count / min_count
        print(f"\nImbalance Ratio: {imbalance_ratio:.2f}")
        
        self.results['class_counts'] = class_counts
        self.results['imbalance_ratio'] = imbalance_ratio
        
        # Visualize
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.bar(range(len(class_counts)), [class_counts[i] for i in sorted(class_counts.keys())])
        plt.xlabel('Class')
        plt.ylabel('Count')
        plt.title('Class Distribution')
        plt.xticks(range(len(self.class_names)), self.class_names, rotation=45, ha='right')
        
        plt.subplot(1, 2, 2)
        plt.pie([class_counts[i] for i in sorted(class_counts.keys())], 
                labels=self.class_names, autopct='%1.1f%%')
        plt.title('Class Distribution (%)')
        
        plt.tight_layout()
        plt.savefig('eda_class_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()
        
    def image_properties(self):
        """Analyze image properties"""
        print("\n🖼️  IMAGE PROPERTIES")
        print("-" * 50)
        
        brightness = np.mean(self.images, axis=(1, 2, 3))
        print(f"Brightness - Mean: {brightness.mean():.2f}, Std: {brightness.std():.2f}")
        
        contrast = np.std(self.images, axis=(1, 2, 3))
        print(f"Contrast - Mean: {contrast.mean():.2f}, Std: {contrast.std():.2f}")
        
        self.results['brightness'] = {'mean': brightness.mean(), 'std': brightness.std()}
        self.results['contrast'] = {'mean': contrast.mean(), 'std': contrast.std()}
        
    def color_analysis(self):
        """Analyze color distributions"""
        print("\n🎨 COLOR ANALYSIS")
        print("-" * 50)
        
        for i, channel in enumerate(['Red', 'Green', 'Blue']):
            channel_mean = np.mean(self.images[:, :, :, i])
            channel_std = np.std(self.images[:, :, :, i])
            print(f"{channel} - Mean: {channel_mean:.2f}, Std: {channel_std:.2f}")
        
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
        colors = ['red', 'green', 'blue']
        channels = ['Red', 'Green', 'Blue']
        
        for i, (ax, color, channel) in enumerate(zip(axes, colors, channels)):
            ax.hist(self.images[:, :, :, i].flatten(), bins=100, color=color, alpha=0.7)
            ax.set_title(f'{channel} Channel Distribution')
            ax.set_xlabel('Pixel Value')
            ax.set_ylabel('Frequency')
        
        plt.tight_layout()
        plt.savefig('eda_color_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()
        
    def texture_analysis(self):
        """Analyze texture features using edge detection"""
        print("\n🔍 TEXTURE ANALYSIS")
        print("-" * 50)
        
        sample_size = min(100, len(self.images))
        edge_intensities = []
        
        for i in range(sample_size):
            gray = cv2.cvtColor(self.images[i], cv2.COLOR_RGB2GRAY)
            edges = cv2.Canny(gray, 50, 150)
            edge_intensity = np.sum(edges) / (edges.shape[0] * edges.shape[1])
            edge_intensities.append(edge_intensity)
        
        print(f"Edge Intensity - Mean: {np.mean(edge_intensities):.4f}, Std: {np.std(edge_intensities):.4f}")
        self.results['edge_intensity'] = {'mean': np.mean(edge_intensities), 'std': np.std(edge_intensities)}
        
    def generate_report(self):
        """Generate comprehensive EDA report"""
        print("\n" + "="*70)
        print("EDA SUMMARY")
        print("="*70)
        
        fig, axes = plt.subplots(len(self.class_names), 5, figsize=(15, 3*len(self.class_names)))
        if len(self.class_names) == 1:
            axes = axes.reshape(1, -1)
        
        for class_idx in range(len(self.class_names)):
            class_images = self.images[self.labels == class_idx][:5]
            for i in range(min(5, len(class_images))):
                if len(class_images) > i:
                    axes[class_idx, i].imshow(class_images[i])
                axes[class_idx, i].axis('off')
                if i == 0:
                    axes[class_idx, i].set_title(f'{self.class_names[class_idx]}', fontsize=10)
        
        plt.suptitle('Sample Images by Class', fontsize=14, y=1.02)
        plt.tight_layout()
        plt.savefig('eda_sample_images.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print("✓ EDA complete. Visualizations saved.")

In [5]:
# ============================================================================
# 2. ADVANCED DATA PREPROCESSING
# ============================================================================

class AdvancedPreprocessor:
    """State-of-the-art preprocessing pipeline"""
    
    def __init__(self, images, labels, img_size=(224, 224)):
        self.images = images
        self.labels = labels
        self.img_size = img_size
        
    def normalize(self, method='imagenet'):
        """Multiple normalization strategies"""
        if method == 'standard':
            self.images = (self.images - np.mean(self.images)) / (np.std(self.images) + 1e-7)
        elif method == 'minmax':
            self.images = (self.images - self.images.min()) / (self.images.max() - self.images.min() + 1e-7)
        elif method == 'imagenet':
            mean = np.array([0.485, 0.456, 0.406]) * 255
            std = np.array([0.229, 0.224, 0.225]) * 255
            self.images = (self.images - mean) / std
        
        print(f"✓ Applied {method} normalization")
        return self.images
    
    def handle_imbalance(self, strategy='class_weights'):
        """Handle class imbalance"""
        if strategy == 'class_weights':
            class_weights = compute_class_weight(
                'balanced',
                classes=np.unique(self.labels),
                y=self.labels
            )
            class_weight_dict = dict(enumerate(class_weights))
            print(f"✓ Computed class weights: {class_weight_dict}")
            return class_weight_dict
        return None

In [6]:
# ============================================================================
# 3. STATE-OF-THE-ART MODEL ARCHITECTURES
# ============================================================================

class ModelFactory:
    """Factory for creating state-of-the-art DL models"""
    
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        
    def create_model(self, model_name, pretrained=True):
        """Create model by name"""
        models_dict = {
            'resnet50': self.resnet50,
            'resnet101': self.resnet101,
            'efficientnetb0': self.efficientnetb0,
            'efficientnetb3': self.efficientnetb3,
            'densenet121': self.densenet121,
            'densenet169': self.densenet169,
            'inceptionv3': self.inceptionv3,
            'mobilenetv2': self.mobilenetv2,
            'xception': self.xception,
            'vgg16': self.vgg16,
            'custom_cnn': self.custom_cnn,
        }
        
        if model_name not in models_dict:
            raise ValueError(f"Model {model_name} not found")
        
        return models_dict[model_name](pretrained)
    
    def _build_head(self, base_model, dropout=0.5):
        """Build classification head"""
        x = layers.GlobalAveragePooling2D()(base_model.output)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(dropout/2)(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        return models.Model(inputs=base_model.input, outputs=outputs)
    
    def resnet50(self, pretrained=True):
        weights = 'imagenet' if pretrained else None
        base = ResNet50(weights=weights, include_top=False, input_shape=self.input_shape)
        return self._build_head(base)
    
    def resnet101(self, pretrained=True):
        weights = 'imagenet' if pretrained else None
        base = ResNet101(weights=weights, include_top=False, input_shape=self.input_shape)
        return self._build_head(base)
    
    def efficientnetb0(self, pretrained=True):
        weights = 'imagenet' if pretrained else None
        base = EfficientNetB0(weights=weights, include_top=False, input_shape=self.input_shape)
        return self._build_head(base)
    
    def efficientnetb3(self, pretrained=True):
        weights = 'imagenet' if pretrained else None
        base = EfficientNetB3(weights=weights, include_top=False, input_shape=self.input_shape)
        return self._build_head(base)
    
    def densenet121(self, pretrained=True):
        weights = 'imagenet' if pretrained else None
        base = DenseNet121(weights=weights, include_top=False, input_shape=self.input_shape)
        return self._build_head(base)
    
    def densenet169(self, pretrained=True):
        weights = 'imagenet' if pretrained else None
        base = DenseNet169(weights=weights, include_top=False, input_shape=self.input_shape)
        return self._build_head(base)
    
    def inceptionv3(self, pretrained=True):
        weights = 'imagenet' if pretrained else None
        base = InceptionV3(weights=weights, include_top=False, input_shape=self.input_shape)
        return self._build_head(base)
    
    def mobilenetv2(self, pretrained=True):
        weights = 'imagenet' if pretrained else None
        base = MobileNetV2(weights=weights, include_top=False, input_shape=self.input_shape)
        return self._build_head(base)
    
    def xception(self, pretrained=True):
        weights = 'imagenet' if pretrained else None
        base = Xception(weights=weights, include_top=False, input_shape=self.input_shape)
        return self._build_head(base)
    
    def vgg16(self, pretrained=True):
        weights = 'imagenet' if pretrained else None
        base = VGG16(weights=weights, include_top=False, input_shape=self.input_shape)
        return self._build_head(base)
    
    def custom_cnn(self, pretrained=False):
        """Custom CNN with residual connections and attention"""
        inputs = layers.Input(shape=self.input_shape)
        
        x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
        
        for filters in [64, 128, 256, 512]:
            shortcut = x
            
            x = layers.Conv2D(filters, 3, padding='same')(x)
            x = layers.BatchNormalization()(x)
            x = layers.Activation('relu')(x)
            x = layers.Conv2D(filters, 3, padding='same')(x)
            x = layers.BatchNormalization()(x)
            
            if shortcut.shape[-1] != filters:
                shortcut = layers.Conv2D(filters, 1, padding='same')(shortcut)
                shortcut = layers.BatchNormalization()(shortcut)
            
            x = layers.Add()([x, shortcut])
            x = layers.Activation('relu')(x)
            x = layers.MaxPooling2D(2)(x)
            
            # Squeeze-and-Excitation block
            se = layers.GlobalAveragePooling2D()(x)
            se = layers.Dense(filters // 16, activation='relu')(se)
            se = layers.Dense(filters, activation='sigmoid')(se)
            se = layers.Reshape((1, 1, filters))(se)
            x = layers.Multiply()([x, se])
        
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(0.3)(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        return models.Model(inputs, outputs, name='Custom_ResNet_Attention')

In [7]:
# ============================================================================
# 4. TRAINING FRAMEWORK - FIXED VERSION
# ============================================================================

class ModelTrainer:
    """Advanced training framework with multiple strategies - FIXED"""
    
    def __init__(self, model, model_name, X_train, y_train, X_val, y_val, class_weights=None):
        self.model = model
        self.model_name = model_name
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.class_weights = class_weights
        self.history = None
        
    def compile_model(self, optimizer='adam', learning_rate=0.001):
        """Compile model with advanced optimizers - FIXED"""
        if optimizer == 'adam':
            opt = optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'adamw':
            opt = optimizers.AdamW(learning_rate=learning_rate)
        elif optimizer == 'sgd':
            opt = optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
        elif optimizer == 'rmsprop':
            opt = optimizers.RMSprop(learning_rate=learning_rate)
        else:
            opt = optimizers.Adam(learning_rate=learning_rate)
        
        # FIX: Use SparseTopKCategoricalAccuracy for sparse labels
        self.model.compile(
            optimizer=opt,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy', 
                    tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name='top3_accuracy')]
        )
    
    def get_callbacks(self, patience=15):
        """Advanced callbacks"""
        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                patience=patience,
                restore_best_weights=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            ModelCheckpoint(
                f'best_{self.model_name}.h5',
                monitor='val_accuracy',
                save_best_only=True,
                verbose=1
            ),
            CSVLogger(f'training_log_{self.model_name}.csv')
        ]
        return callbacks
    
    def train(self, epochs=100, batch_size=32, patience=15):
        """Train model"""
        print(f"\n{'='*70}")
        print(f"Training {self.model_name}")
        print(f"{'='*70}")
        
        self.history = self.model.fit(
            self.X_train, self.y_train,
            validation_data=(self.X_val, self.y_val),
            epochs=epochs,
            batch_size=batch_size,
            class_weight=self.class_weights,
            callbacks=self.get_callbacks(patience=patience),
            verbose=1
        )
        
        return self.history
    
    def evaluate(self):
        """Comprehensive evaluation"""
        y_pred_probs = self.model.predict(self.X_val, verbose=0)
        y_pred = np.argmax(y_pred_probs, axis=1)
        
        results = {
            'accuracy': accuracy_score(self.y_val, y_pred),
            'predictions': y_pred,
            'probabilities': y_pred_probs,
            'history': self.history.history if self.history else None
        }
        
        return results

In [8]:
# ============================================================================
# 5. ENSEMBLE METHODS
# ============================================================================

class EnsembleModel:
    """Advanced ensemble methods"""
    
    def __init__(self, models, model_names):
        self.models = models
        self.model_names = model_names
        
    def predict_voting(self, X, method='soft'):
        """Voting ensemble"""
        predictions = []
        for model in self.models:
            if method == 'soft':
                predictions.append(model.predict(X, verbose=0))
            else:
                predictions.append(np.argmax(model.predict(X, verbose=0), axis=1))
        
        if method == 'soft':
            avg_predictions = np.mean(predictions, axis=0)
            return np.argmax(avg_predictions, axis=1), avg_predictions
        else:
            votes = np.array(predictions).T
            return np.apply_along_axis(lambda x: np.bincount(x).argmax(), 1, votes)

In [9]:
# ============================================================================
# 6. COMPREHENSIVE EVALUATION AND VISUALIZATION
# ============================================================================

class ModelEvaluator:
    """Comprehensive model evaluation and comparison"""
    
    def __init__(self, class_names):
        self.class_names = class_names
        self.results = {}
        
    def evaluate_model(self, model_name, y_true, y_pred, y_pred_probs, history=None):
        """Evaluate single model"""
        results = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)[0],
            'recall': precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)[1],
            'f1_score': precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)[2],
            'cohen_kappa': cohen_kappa_score(y_true, y_pred),
            'matthews_corrcoef': matthews_corrcoef(y_true, y_pred),
            'confusion_matrix': confusion_matrix(y_true, y_pred),
            'classification_report': classification_report(y_true, y_pred, target_names=self.class_names, zero_division=0),
            'history': history
        }
        
        # ROC AUC for multi-class
        try:
            from sklearn.preprocessing import label_binarize
            y_true_bin = label_binarize(y_true, classes=range(len(self.class_names)))
            results['roc_auc'] = roc_auc_score(y_true_bin, y_pred_probs, average='weighted', multi_class='ovr')
        except:
            results['roc_auc'] = None
        
        self.results[model_name] = results
        return results
    
    def print_results(self, model_name):
        """Print detailed results"""
        results = self.results[model_name]
        print(f"\n{'='*70}")
        print(f"EVALUATION RESULTS: {model_name}")
        print(f"{'='*70}")
        print(f"Accuracy: {results['accuracy']:.4f}")
        print(f"Precision: {results['precision']:.4f}")
        print(f"Recall: {results['recall']:.4f}")
        print(f"F1-Score: {results['f1_score']:.4f}")
        print(f"Cohen's Kappa: {results['cohen_kappa']:.4f}")
        print(f"Matthews Correlation: {results['matthews_corrcoef']:.4f}")
        if results['roc_auc']:
            print(f"ROC AUC: {results['roc_auc']:.4f}")
        print(f"\n{results['classification_report']}")
    
    def plot_confusion_matrix(self, model_name):
        """Plot confusion matrix"""
        cm = self.results[model_name]['confusion_matrix']
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=self.class_names, yticklabels=self.class_names)
        plt.title(f'Confusion Matrix - {model_name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.savefig(f'confusion_matrix_{model_name}.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    def plot_training_history(self, model_name):
        """Plot training history"""
        history = self.results[model_name]['history']
        if history is None:
            return
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Accuracy
        axes[0].plot(history['accuracy'], label='Train', linewidth=2)
        axes[0].plot(history['val_accuracy'], label='Validation', linewidth=2)
        axes[0].set_title(f'Model Accuracy - {model_name}', fontsize=14)
        axes[0].set_xlabel('Epoch', fontsize=12)
        axes[0].set_ylabel('Accuracy', fontsize=12)
        axes[0].legend(fontsize=11)
        axes[0].grid(True, alpha=0.3)
        
        # Loss
        axes[1].plot(history['loss'], label='Train', linewidth=2)
        axes[1].plot(history['val_loss'], label='Validation', linewidth=2)
        axes[1].set_title(f'Model Loss - {model_name}', fontsize=14)
        axes[1].set_xlabel('Epoch', fontsize=12)
        axes[1].set_ylabel('Loss', fontsize=12)
        axes[1].legend(fontsize=11)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'training_history_{model_name}.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    def compare_models(self):
        """Compare all models"""
        if not self.results:
            print("⚠ No models to compare")
            return None
            
        print(f"\n{'='*70}")
        print("MODEL COMPARISON")
        print(f"{'='*70}")
        
        comparison_df = pd.DataFrame({
            'Model': list(self.results.keys()),
            'Accuracy': [r['accuracy'] for r in self.results.values()],
            'Precision': [r['precision'] for r in self.results.values()],
            'Recall': [r['recall'] for r in self.results.values()],
            'F1-Score': [r['f1_score'] for r in self.results.values()],
            'Cohen Kappa': [r['cohen_kappa'] for r in self.results.values()],
            'Matthews Corr': [r['matthews_corrcoef'] for r in self.results.values()],
        })
        
        comparison_df = comparison_df.sort_values('Accuracy', ascending=False)
        print(comparison_df.to_string(index=False))
        
        comparison_df.to_csv('model_comparison.csv', index=False)
        
        # Visualization
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Cohen Kappa', 'Matthews Corr']
        
        for idx, (ax, metric) in enumerate(zip(axes.flat, metrics)):
            data = comparison_df.sort_values(metric, ascending=True)
            ax.barh(data['Model'], data[metric])
            ax.set_xlabel(metric, fontsize=11)
            ax.set_title(f'{metric} Comparison', fontsize=12)
            ax.grid(True, alpha=0.3, axis='x')
        
        plt.tight_layout()
        plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        return comparison_df

In [10]:
# ========== CONFIGURATION ==========
DATA_PATH = "/kaggle/input/comprehensive-soil-classification-datasets/CyAUG-Dataset"

# Choose: 'all', 'fast', or custom list
MODELS_TO_TRAIN = 'fast'  # Fast models for quick testing

CONFIG = {
    'img_size': (224, 224),
    'batch_size': 32,
    'epochs': 50,
    'patience': 10,  # Early stopping patience
    'learning_rate': 0.001,
    'validation_split': 0.2,
    'test_split': 0.2,
    'optimizer': 'adam',
    'normalization': 'imagenet',
}

print("\n" + "="*80)
print("COMPREHENSIVE SOIL CLASSIFICATION PIPELINE - FIXED VERSION")
print("="*80)
print(f"\n📋 Configuration:")
print(f"   Data Path: {DATA_PATH}")
print(f"   Models: {MODELS_TO_TRAIN}")
print(f"   Image Size: {CONFIG['img_size']}")
print(f"   Batch Size: {CONFIG['batch_size']}")
print(f"   Epochs: {CONFIG['epochs']}")
print(f"   Patience: {CONFIG['patience']}")


COMPREHENSIVE SOIL CLASSIFICATION PIPELINE - FIXED VERSION

📋 Configuration:
   Data Path: /kaggle/input/comprehensive-soil-classification-datasets/CyAUG-Dataset
   Models: fast
   Image Size: (224, 224)
   Batch Size: 32
   Epochs: 50
   Patience: 10


In [11]:
# ========== VERIFY DATA PATH ==========
data_path = Path(DATA_PATH)

if not data_path.exists():
    print(f"\n❌ ERROR: Data path not found: {DATA_PATH}")
    print("Please update DATA_PATH variable")
    exit(1)

soil_types = [d.name for d in data_path.iterdir() if d.is_dir()]
print(f"\n✓ Found {len(soil_types)} soil types:")
for soil_type in sorted(soil_types)[:10]:  # Show first 10
    num_images = len(list((data_path / soil_type).glob('*.jpg'))) + \
                    len(list((data_path / soil_type).glob('*.png')))
    print(f"   - {soil_type}: {num_images} images")


✓ Found 7 soil types:
   - Alluvial_Soil: 49 images
   - Arid_Soil: 284 images
   - Black_Soil: 255 images
   - Laterite_Soil: 219 images
   - Mountain_Soil: 201 images
   - Red_Soil: 108 images
   - Yellow_Soil: 69 images


In [12]:
# ========== CREATE OUTPUT DIRECTORY ==========
output_dir = Path('soil_classification_results')
output_dir.mkdir(exist_ok=True)
os.chdir(output_dir)
print(f"\n✓ Output directory: {output_dir.absolute()}")


✓ Output directory: /kaggle/working/soil_classification_results/soil_classification_results


In [13]:
# Step 1: Load Data
print("\n" + "="*80)
print("STEP 1: DATA LOADING")
print("="*80)

loader = SoilDataLoader(data_path)
images, labels, class_names = loader.load_image_directory(img_size=CONFIG['img_size'])


STEP 1: DATA LOADING
✓ Found 7 classes: ['Alluvial_Soil', 'Arid_Soil', 'Black_Soil', 'Laterite_Soil', 'Mountain_Soil', 'Red_Soil', 'Yellow_Soil']
   Loading Alluvial_Soil: 692 images...
   Loading Arid_Soil: 284 images...
   Loading Black_Soil: 1173 images...
   Loading Laterite_Soil: 219 images...
   Loading Mountain_Soil: 201 images...
   Loading Red_Soil: 1126 images...
   Loading Yellow_Soil: 1401 images...
✓ Successfully loaded 5096 images


In [14]:
# Step 2: EDA
print("\n" + "="*80)
print("STEP 2: EXPLORATORY DATA ANALYSIS")
print("="*80)

eda = SoilEDA(images, labels, class_names)
eda.analyze_all()


STEP 2: EXPLORATORY DATA ANALYSIS

EXPLORATORY DATA ANALYSIS

📊 BASIC STATISTICS
--------------------------------------------------
Total samples: 5096
Number of classes: 7
Image shape: (224, 224, 3)
Data type: uint8
Value range: [0, 255]

📈 CLASS DISTRIBUTION
--------------------------------------------------
Alluvial_Soil: 692 (13.58%)
Arid_Soil: 284 (5.57%)
Black_Soil: 1173 (23.02%)
Laterite_Soil: 219 (4.30%)
Mountain_Soil: 201 (3.94%)
Red_Soil: 1126 (22.10%)
Yellow_Soil: 1401 (27.49%)

Imbalance Ratio: 6.97

🖼️  IMAGE PROPERTIES
--------------------------------------------------
Brightness - Mean: 104.19, Std: 41.28
Contrast - Mean: 54.17, Std: 19.18

🎨 COLOR ANALYSIS
--------------------------------------------------
Red - Mean: 149.03, Std: 70.33
Green - Mean: 100.95, Std: 59.91
Blue - Mean: 62.59, Std: 52.28

🔍 TEXTURE ANALYSIS
--------------------------------------------------
Edge Intensity - Mean: 80.6087, Std: 17.9521

EDA SUMMARY
✓ EDA complete. Visualizations saved.


In [15]:
# Step 3: Preprocessing
print("\n" + "="*80)
print("STEP 3: DATA PREPROCESSING")
print("="*80)

preprocessor = AdvancedPreprocessor(images, labels)
images = preprocessor.normalize(method=CONFIG['normalization'])
class_weights = preprocessor.handle_imbalance()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, 
    test_size=CONFIG['test_split'], 
    stratify=labels, 
    random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, 
    test_size=CONFIG['validation_split'], 
    stratify=y_train, 
    random_state=42
)

print(f"\n✓ Dataset Split:")
print(f"   Train: {len(X_train)} ({len(X_train)/len(images)*100:.1f}%)")
print(f"   Validation: {len(X_val)} ({len(X_val)/len(images)*100:.1f}%)")
print(f"   Test: {len(X_test)} ({len(X_test)/len(images)*100:.1f}%)")


STEP 3: DATA PREPROCESSING
✓ Applied imagenet normalization
✓ Computed class weights: {0: 1.0520231213872833, 1: 2.563380281690141, 2: 0.6206308610400681, 3: 3.324200913242009, 4: 3.6218905472636815, 5: 0.6465364120781527, 6: 0.5196288365453248}

✓ Dataset Split:
   Train: 3260 (64.0%)
   Validation: 816 (16.0%)
   Test: 1020 (20.0%)


In [16]:
# Step 4: Model Training
print("\n" + "="*80)
print("STEP 4: MODEL TRAINING")
print("="*80)

if MODELS_TO_TRAIN == 'all':
    model_list = ['mobilenetv2', 'efficientnetb0', 'resnet50', 'densenet121', 'inceptionv3', 'xception', 'custom_cnn']
elif MODELS_TO_TRAIN == 'fast':
    model_list = ['mobilenetv2', 'efficientnetb0', 'resnet50']
else:
    model_list = MODELS_TO_TRAIN

print(f"\n📋 Models to train: {', '.join(model_list)}")

factory = ModelFactory(input_shape=images.shape[1:], num_classes=len(class_names))
trained_models = {}
evaluator = ModelEvaluator(class_names)

for idx, model_name in enumerate(model_list, 1):
    try:
        print(f"\n{'='*80}")
        print(f"[{idx}/{len(model_list)}] TRAINING: {model_name.upper()}")
        print(f"{'='*80}")
        
        model = factory.create_model(model_name, pretrained=True)
        print(f"✓ Model created: {model.count_params():,} parameters")
        
        trainer = ModelTrainer(model, model_name, X_train, y_train, X_val, y_val, class_weights)
        trainer.compile_model(optimizer=CONFIG['optimizer'], learning_rate=CONFIG['learning_rate'])
        trainer.train(epochs=CONFIG['epochs'], batch_size=CONFIG['batch_size'], patience=CONFIG['patience'])
        
        results = trainer.evaluate()
        evaluator.evaluate_model(model_name, y_val, results['predictions'],
                                results['probabilities'], results['history'])
        
        evaluator.print_results(model_name)
        evaluator.plot_confusion_matrix(model_name)
        evaluator.plot_training_history(model_name)
        
        trained_models[model_name] = model
        print(f"\n✓ {model_name} training complete!")
        
    except Exception as e:
        print(f"\n❌ Error training {model_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue


STEP 4: MODEL TRAINING

📋 Models to train: mobilenetv2, efficientnetb0, resnet50

[1/3] TRAINING: MOBILENETV2


I0000 00:00:1760268673.994263      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✓ Model created: 3,054,151 parameters

Training mobilenetv2
Epoch 1/50


I0000 00:00:1760268731.091696      62 service.cc:148] XLA service 0x78209c0036d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760268731.092564      62 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1760268735.540924      62 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1760268742.917865      62 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1760268743.114765      62 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


  1/102 ━━━━━━━━━━━━━━━━━━━━ 2:08:51 77s/step - accuracy: 0.0938 - loss: 2.2886 - top3_accuracy: 0.4375

...

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.5775 - loss: 1.3179 - top3_accuracy: 0.8024
Epoch 1: val_accuracy improved from -inf to 0.40931, saving model to best_mobilenetv2.h5
102/102 ━━━━━━━━━━━━━━━━━━━━ 125s 477ms/step - accuracy: 0.5789 - loss: 1.3150 - top3_accuracy: 0.8034 - val_accuracy: 0.4093 - val_loss: 5.7818 - val_top3_accuracy: 0.8015 - learning_rate: 0.0010
Epoch 2/50
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.8384 - loss: 0.6397 - top3_accuracy: 0.9562
Epoch 2: val_accuracy improved from 0.40931 to 0.61765, saving model to best_mobilenetv2.h5
102/102 ━━━━━━━━━━━━━━━━━━━━ 10s 96ms/step - accuracy: 0.8384 - loss: 0.6395 - top3_accuracy: 0.9562 - val_accuracy: 0.6176 - val_loss: 8.7473 - val_top3_accuracy: 0.7255 - learning_rate: 0.0010
Epoch 3/50
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.8785 - loss: 0.4546 - top3_accuracy: 0.9703
Epoch 3: val_accuracy did not improve from 0.61765
102/102 ━━━━━━━━━━━━━━━━━━━━ 9s 91ms/step - accuracy:

E0000 00:00:1760269069.704521      62 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1760269069.894071      62 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1760269070.390197      62 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1760269070.596302      62 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1760269070.971085      62 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

101/102 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.6324 - loss: 1.1823 - top3_accuracy: 0.8228

E0000 00:00:1760269123.575316      63 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1760269123.765331      63 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1760269124.246386      63 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1760269124.455364      63 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1760269124.828607      63 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 525ms/step - accuracy: 0.6337 - loss: 1.1790 - top3_accuracy: 0.8238
Epoch 1: val_accuracy improved from -inf to 0.23897, saving model to best_efficientnetb0.h5
102/102 ━━━━━━━━━━━━━━━━━━━━ 185s 714ms/step - accuracy: 0.6350 - loss: 1.1758 - top3_accuracy: 0.8248 - val_accuracy: 0.2390 - val_loss: 2.4285 - val_top3_accuracy: 0.4326 - learning_rate: 0.0010
Epoch 2/50
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.8661 - loss: 0.5333 - top3_accuracy: 0.9683
Epoch 2: val_accuracy improved from 0.23897 to 0.29167, saving model to best_efficientnetb0.h5
102/102 ━━━━━━━━━━━━━━━━━━━━ 14s 135ms/step - accuracy: 0.8662 - loss: 0.5328 - top3_accuracy: 0.9683 - val_accuracy: 0.2917 - val_loss: 2.1198 - val_top3_accuracy: 0.5196 - learning_rate: 0.0010
Epoch 3/50
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.9203 - loss: 0.2914 - top3_accuracy: 0.9845
Epoch 3: val_accuracy did not improve from 0.29167
102/102 ━━━━━━━━━━━━━━━━━━━━ 13s 126ms/step 

In [17]:
# Step 5: Ensemble (if multiple models)
if len(trained_models) >= 2:
    print("\n" + "="*80)
    print("STEP 5: ENSEMBLE METHODS")
    print("="*80)
    
    ensemble = EnsembleModel(list(trained_models.values()), list(trained_models.keys()))
    y_pred_ensemble, y_pred_probs_ensemble = ensemble.predict_voting(X_val, method='soft')
    evaluator.evaluate_model('Ensemble_Voting', y_val, y_pred_ensemble, y_pred_probs_ensemble)
    evaluator.print_results('Ensemble_Voting')
    evaluator.plot_confusion_matrix('Ensemble_Voting')
    print("✓ Ensemble evaluation complete!")


STEP 5: ENSEMBLE METHODS

EVALUATION RESULTS: Ensemble_Voting
Accuracy: 0.9779
Precision: 0.9795
Recall: 0.9779
F1-Score: 0.9778
Cohen's Kappa: 0.9724
Matthews Correlation: 0.9725
ROC AUC: 0.9993

               precision    recall  f1-score   support

Alluvial_Soil       0.97      0.98      0.98       111
    Arid_Soil       0.93      0.89      0.91        46
   Black_Soil       1.00      0.99      1.00       188
Laterite_Soil       0.96      0.77      0.86        35
Mountain_Soil       0.80      1.00      0.89        32
     Red_Soil       0.99      1.00      0.99       180
  Yellow_Soil       1.00      0.99      0.99       224

     accuracy                           0.98       816
    macro avg       0.95      0.95      0.95       816
 weighted avg       0.98      0.98      0.98       816

✓ Ensemble evaluation complete!


In [18]:
# Step 6: Test Set Evaluation
print("\n" + "="*80)
print("STEP 6: FINAL TEST SET EVALUATION")
print("="*80)

if trained_models:
    best_model_name = max(evaluator.results.items(), key=lambda x: x[1]['accuracy'])[0]
    best_model = trained_models.get(best_model_name)
    
    if best_model:
        print(f"\n🏆 Best Model: {best_model_name}")
        
        y_test_pred_probs = best_model.predict(X_test, verbose=0)
        y_test_pred = np.argmax(y_test_pred_probs, axis=1)
        
        test_accuracy = accuracy_score(y_test, y_test_pred)
        print(f"📊 Test Accuracy: {test_accuracy:.4f}")
        print(f"\n{classification_report(y_test, y_test_pred, target_names=class_names, zero_division=0)}")
        
        # Test confusion matrix
        cm_test = confusion_matrix(y_test, y_test_pred)
        plt.figure(figsize=(12, 10))
        sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names, yticklabels=class_names)
        plt.title(f'Test Confusion Matrix - {best_model_name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('test_confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.close()
else:
    print("\n⚠ No models were successfully trained")
    best_model = None
    best_model_name = "None"
    test_accuracy = 0.0


STEP 6: FINAL TEST SET EVALUATION


In [19]:
# Step 7: Comparison
print("\n" + "="*80)
print("STEP 7: MODEL COMPARISON")
print("="*80)

comparison_df = evaluator.compare_models()


STEP 7: MODEL COMPARISON

MODEL COMPARISON
          Model  Accuracy  Precision   Recall  F1-Score  Cohen Kappa  Matthews Corr
Ensemble_Voting  0.977941   0.979483 0.977941  0.977779     0.972353       0.972467
 efficientnetb0  0.974265   0.974307 0.974265  0.973544     0.967698       0.967794
       resnet50  0.960784   0.962216 0.960784  0.961353     0.950951       0.951001
    mobilenetv2  0.764706   0.851243 0.764706  0.773751     0.710675       0.729998


In [20]:
# Step 8: Save Results
print("\n" + "="*80)
print("STEP 8: SAVING RESULTS")
print("="*80)

if trained_models and best_model:
    best_model.save(f'final_best_model_{best_model_name}.h5')
    print(f"✓ Saved: final_best_model_{best_model_name}.h5")

results_summary = {
    'best_model': best_model_name if trained_models else 'None',
    'test_accuracy': float(test_accuracy) if trained_models and best_model else 0.0,
    'configuration': CONFIG,
    'class_names': class_names,
    'num_classes': len(class_names),
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

with open('results_summary.json', 'w') as f:
    json.dump(results_summary, f, indent=4)
print("✓ Saved: results_summary.json")


STEP 8: SAVING RESULTS
✓ Saved: results_summary.json


In [21]:
# Final Summary
print("\n" + "="*80)
print("✅ PIPELINE COMPLETE!")
print("="*80)
if trained_models and best_model:
    print(f"\n🏆 Best Model: {best_model_name}")
    print(f"📊 Test Accuracy: {test_accuracy:.4f}")
    print(f"\n📁 All results saved in: {output_dir.absolute()}")
else:
    print("\n⚠ No models were successfully trained. Check errors above.")
    print(f"📁 Available outputs in: {output_dir.absolute()}")


✅ PIPELINE COMPLETE!

⚠ No models were successfully trained. Check errors above.
📁 Available outputs in: /kaggle/working/soil_classification_results/soil_classification_results
